Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 3: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside the course repo 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.



In [64]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy
import numpy as np
import math


## The *Nim* and *Nimply* classes

In [65]:
Nimply = namedtuple("Nimply", "row, num_objects")


In [66]:
class Nim:
    def __init__(self, num_rows: int) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        self._rows[row] -= num_objects


## Sample (and silly) startegies 

In [67]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)


In [68]:
def take_from_last(state: Nim) -> Nimply:
    """Takes a random number from the last row"""
    row = max([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)

In [69]:
def take_from_first(state: Nim) -> Nimply:
    """Takes a random number from the first row"""
    row = min([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)

In [70]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (m[0], m[1])))


In [71]:
def eleirbag(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the highest row, the opposite of gabriele"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))

In [72]:
import numpy as np


def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)


def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = random.choice(spicy_moves)
    return ply


## Oversimplified match

In [73]:
logging.getLogger().setLevel(logging.INFO)

strategy = (gabriele, optimal)

nim = Nim(5)
logging.info(f"init : {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    logging.info(f"ply: player {player} plays {ply}")
    nim.nimming(ply)
    logging.info(f"status: {nim}")
    player = 1 - player
logging.info(f"status: Player {player} won!")

INFO:root:init : <1 3 5 7 9>
INFO:root:ply: player 0 plays Nimply(row=4, num_objects=9)
INFO:root:status: <1 3 5 7 0>
INFO:root:ply: player 1 plays Nimply(row=2, num_objects=1)
INFO:root:status: <1 3 4 7 0>
INFO:root:ply: player 0 plays Nimply(row=3, num_objects=7)
INFO:root:status: <1 3 4 0 0>
INFO:root:ply: player 1 plays Nimply(row=0, num_objects=1)
INFO:root:status: <0 3 4 0 0>
INFO:root:ply: player 0 plays Nimply(row=2, num_objects=4)
INFO:root:status: <0 3 0 0 0>
INFO:root:ply: player 1 plays Nimply(row=1, num_objects=1)
INFO:root:status: <0 2 0 0 0>
INFO:root:ply: player 0 plays Nimply(row=1, num_objects=2)
INFO:root:status: <0 0 0 0 0>
INFO:root:status: Player 1 won!


In [74]:
class adaptive_class:
    def __init__(self, stdev = 0.2, stdev_step = 0.1) -> None:
        self.stdev = stdev
        self.stdev_step = stdev_step
        self.genome = {optimal: 0, pure_random: 0, take_from_last: 0, take_from_first: 0, eleirbag: 0}
        for k in self.genome.keys():
            self.genome[k] = 1

    def tweak(self) -> None:
        for k in self.genome.keys():
            self.genome[k] += np.random.normal(0, self.stdev) 
            self.genome[k] = 0 if self.genome[k] < 0 else self.genome[k] 

    def apply_rule(self, ratio) -> None:
        self.stdev += self.stdev_step * (ratio - 0.2)

    def adaptive(self, state: Nim) -> Nimply:
        """A strategy that can adapt its parameters"""
        a_random = random.random() * sum(self.genome.values())
        k = pure_random
        for k in self.genome.keys():
            a_random -= self.genome[k]
            if a_random <= 0:
                break 
        return k(state)  

adaptive_obj = adaptive_class()
adaptive_obj.adaptive(Nim(5))
print(adaptive_obj.genome)
adaptive_obj.tweak()
print(adaptive_obj.genome)

{<function optimal at 0x7ff8a0042050>: 1, <function pure_random at 0x7ff87d0a28c0>: 1, <function take_from_last at 0x7ff87d0a2cb0>: 1, <function take_from_first at 0x7ff87d0a2c20>: 1, <function eleirbag at 0x7ff87d0a2d40>: 1}
{<function optimal at 0x7ff8a0042050>: 1.1609229959549923, <function pure_random at 0x7ff87d0a28c0>: 1.015478287979048, <function take_from_last at 0x7ff87d0a2cb0>: 0.6009595187581902, <function take_from_first at 0x7ff87d0a2c20>: 1.0350592914958952, <function eleirbag at 0x7ff87d0a2d40>: 0.6791584646960112}


In [75]:
def play_game(strategy: tuple) -> int:
    """plays a game of nim(5) and returns the winning player index"""
    nim = Nim(5)
    player = 0
    while nim:
        ply = strategy[player](nim)
        nim.nimming(ply)
        player = 1 - player
    return player
    

In [77]:
def getratio(adaptive_obj: adaptive_class) -> float: 
    """return winning ratio of an adaptive strategy playing 100 games vs gabriele"""
    wins = [0, 0]
    strategy = (adaptive_obj.adaptive, gabriele)
    for _ in range(50):
        wins[play_game(strategy)] += 1
    strategy = (gabriele, adaptive_obj.adaptive) 
    for _ in range(50):
        wins[1 - play_game(strategy)] += 1   
    return wins[0] / wins[1]

## Adapting the strategy
 how does it work:
  - we create 10 copies of the original adaptive object
  - for each copy, we play 100 games against gabriele (50 going first, 50 going second)
  - we measure the winning rate = wins / total games
  - we measure how many are better to apply the 1/5 rule
  - we take the best among the 10 and tweak the stdev based on 1/5 rule
  - repeat many times and see the result

In [79]:
adaptive_obj = adaptive_class()
ratio_prev = getratio(adaptive_obj)
print(ratio_prev)

1.7027027027027026
